# Tool Usage

Language models are best suited for generating natural language. They don't have access to external knowledge, and may not be well suited to for computational tasks. However, we can overcome some of these limits by augmenting models with tools.

## Prompting for Tool Use

The first step is to prompt the model in a way that allows it make use of tools. We'll do this by providing few-shot examples of computations using eval. We can then replace these computations with their results.

In [1]:
import languagemodels as lm
import re

In [2]:
def generate_answer_for_calculator(question):
    return lm.do(f"""
Answer using eval as needed.

Question: I had 17 apples and get 8 more. How many apples do I have?
Answer: You have eval(17 + 8) apples.

Question: How many dogs do I have if I start with 3 and get 2 more?
Answer: You have eval(3 + 2) dogs.

Question: I had 211 books and lose 154, how many books do I have?
Answer: You have eval(211 - 154) books.

Question: If I had 253 cats and got 101 more, how many cats do I have?
Answer: You have eval(253 + 101) cats.

Question: I buy 6 oranges and had 4 to begin with. How many oranges do I have?
Answer: You have eval(6 + 4) oranges.

Question: {question}
""".strip())

reply = generate_answer_for_calculator("If I have 28 cars and buy 51 more, how many cars do I have?")
reply

'You have eval(28 + 51) cars.'

## Merging Tools and Results

Now that we have a result from the LLM expecting tools to be used, we can use regular expressions to replace tools with their results.

In [3]:
def replace_expressions(reply):
    # Replace "eval(1+2)" with 3
    # Also replace "eval(1+2) = 3" with 3, as the model sometimes predicts an answer
    expressions = re.findall('(eval\(([ 0-9\.+\-/\*]+)\)[ =0-9\.]*)', reply)

    for exp in expressions:
        result = eval(exp[1])
        reply = reply.replace(exp[0].strip(), str(result))
        
    return reply

replace_expressions(reply)

'You have 79 cars.'